In [3]:
import numpy as np

In [46]:
from count_freqs import * 
counter = Hmm(3)
input = open("gene_mod.train","r")
counter.train(input)
output=open("original_counts","w")
counter.write_counts(output)
output.close()

In [45]:
trained=open("original_counts","r")
x=trained.readlines()
trained.close()
words={}
for i in range(len(x)):
    x[i]=x[i].split()
    if x[i][1]=='WORDTAG':
        if x[i][3] not in words.keys():
            words[x[i][3]]=int(x[i][0])
        else:
            words[x[i][3]]+=int(x[i][0])
rem_words=[]
for i in words.keys():
    if words[i]==1:
        rem_words.append(i)

In [36]:
def replace(filein, words, fileout):
    f = open(filein,'r')
    f2 = open(fileout,'w')
    filedata = f.readlines()
    f.close()
    
    for i in range(len(filedata)):
        x=filedata[i].split()
        for word in rem_words:
            if word in x and 'I-GENE' not in x:
                idx=x.index(word)
                x[idx]='_RARE_'
        f2.write(' '.join(x))
        f2.write('\n')
    
    f2.close()
    
replace("gene.train", words, "gene_mod1.train")

In [4]:
trained=open("trial1","r")
x=trained.readlines()
trained.close()
all_words=[]
for i in range(len(x)):
    x[i]=x[i].split()
    if x[i][1]=='WORDTAG':
        all_words.append(x[i][3])

In [8]:
from count_freqs import * 
counter_dev = Hmm(3)
input = open("gene.dev","r")
counter_dev.train(input)
output=open("count_dev","w")
counter_dev.write_counts(output)
output.close()

In [10]:
dev=open("count_dev","r")
x=dev.readlines()
dev.close()
dev_words=[]
for i in range(len(x)):
    x[i]=x[i].split()
    if x[i][1]=='WORDTAG':
        dev_words.append(x[i][2])

In [ ]:
dev = open("gene.dev","r")
y=dev.readlines()
dev.close()

dev_test=open("gene_dev_pred","w")

for i in y:
    i=i[:-1]
    if i in all_words:
        e_gene=emission_parameters(x, i, 'I-GENE')
        e_o=emission_parameters(x, i, 'O')
        if e_gene> e_o:
            i = i + ' I-GENE\n'
        else:
            i = i + ' O\n' 
    elif len(i)==0:
        i='\n'
    else:
        i = i + ' O\n'
    dev_test.write(i)
dev_test.close()

In [16]:
dev = open("gene.dev","r")
y=dev.readlines()
dev.close()

In [4]:
from eval_gene_tagger import *
f1=open("gene.key","r")
f2=open("gene_dev_try","r")
gs_iterator = corpus_iterator(f1)
pred_iterator = corpus_iterator(f2, with_logprob = False)
evaluator = Evaluator()
evaluator.compare(gs_iterator, pred_iterator)
evaluator.print_scores()
f1.close();f2.close()

Found 506 GENEs. Expected 642 GENEs; Correct: 234.

	 precision 	recall 		F1-Score
GENE:	 0.462451	0.364486	0.407666


In [19]:
from eval_gene_tagger import *
f1=open("gene.train","r")
f2=open("gene_train_try1","r")
gs_iterator = corpus_iterator(f1)
pred_iterator = corpus_iterator(f2, with_logprob = False)
evaluator = Evaluator()
evaluator.compare(gs_iterator, pred_iterator)
evaluator.print_scores()
f1.close();f2.close()

Found 15527 GENEs. Expected 16637 GENEs; Correct: 6416.

	 precision 	recall 		F1-Score
GENE:	 0.413216	0.385646	0.398955


Modifications to rare classes

In [10]:
dev = open("gene.dev","r")
y=dev.readlines()
dev.close()

dev_test=open("gene_dev_pred1","w")

for i in y:
    i=i[:-1]
    if i in all_words:
        e_gene=emission_parameters(x, i, 'I-GENE')
        e_o=emission_parameters(x, i, 'O')
        if e_gene> e_o:
            i = i + ' I-GENE\n'
        else:
            i = i + ' O\n' 
    elif len(i)==0:
        i='\n'
    else:
        if i.isupper() or i.endswith('ic') or i.endswith('ase')or i.endswith('in')or i.endswith('ic')or i.endswith('ica')or i.endswith('gen')or i.endswith('ases')or i.endswith('or')or i.endswith('ors')or i.endswith('ory'):
                i = i + ' I-GENE\n'
        else:
            i = i + ' O\n'
    dev_test.write(i)
dev_test.close()

In [4]:
train = open("gene.train","r")
y=train.readlines()
train.close()

train_test=open("gene_train_pred1","w")

for i in y:
    if i!='\n':
        i=i.split()[0]
    if i in all_words:
        e_gene=emission_parameters(x, i, 'I-GENE')
        e_o=emission_parameters(x, i, 'O')
        if e_gene> e_o:
            i = i + ' I-GENE\n'
        else:
            i = i + ' O\n' 
    elif i=='\n':
        i='\n'
    else:
        if i.isupper() or i.endswith('ic') or i.endswith('ase')or i.endswith('in')or i.endswith('ic')or i.endswith('ica')or i.endswith('gen')or i.endswith('ases')or i.endswith('or')or i.endswith('ors')or i.endswith('ory'):
                i = i + ' I-GENE\n'
        else:
            i = i + ' O\n'
    train_test.write(i)
train_test.close()

In [5]:
def emission_parameters(x, word, tag):
    den=0;num=0
    if word not in all_words:
        i=word
        if i.isupper() or i.endswith('ic') or i.endswith('ase')or i.endswith('in')or i.endswith('ic')or i.endswith('ica')or i.endswith('gen')or i.endswith('ases')or i.endswith('or')or i.endswith('ors')or i.endswith('ory'):
            word='-'
        else:
            word='_RARE_'
    for i in range(len(x)):
        if x[i][1]=='WORDTAG' and x[i][2]==tag:
            if x[i][3]==word:
                num=int(x[i][0])
            den+=int(x[i][0])        
    return num/den
     

In [6]:
def transmission_parameters(x, y2, y1, y):
    num=0;den=0
    for i in x[20139:]:
        if i[1]=='3-GRAM' and i[2]== y2 and i[3]==y1 and i[4]== y:
            num=int(i[0])
        if i[1]=='2-GRAM' and i[2]== y2 and i[3]==y1:
            den=int(i[0])
    return num/den    

In [5]:
def transmission_parameters_bigram(x, y1, y):
    num=0;den=0
    for i in x[20139:]:
        if i[1]=='2-GRAM' and i[2]== y1 and i[3]==y: 
            num=int(i[0])
        if i[1]=='1-GRAM' and i[2]== y1:
            den=int(i[0])
    return num/den

In [7]:
def backtrace(z_pred, back, n, labels):
    prev1, prev2 = z_pred[-1], z_pred[-2]
    for k in np.arange(n-2)[::-1]:
        z_pred[k] = back[prev2, prev1,k+2]
    return [labels[z] for z in z_pred] 
        
def viterbi(sent):
    #0- tag 'O'
    #1- tag 'I-GENE'
    n=len(sent)
    p = np.zeros(shape=(2, 2, n))
    back = np.zeros(shape=(2, 2, n), dtype=np.int)
    z_pred = np.zeros(shape=(n, ), dtype=np.int)
    tag=['O','I-GENE']
    for q in range(len(tag)):
        temp=transmission_parameters(x, '*', '*', tag[q]) * emission_parameters(x, sent[0], tag[q])
        p[q, 0, 0] = temp
        p[q, 1, 0] = temp
        back[q, 0, 0] = 0
        back[q, 1, 0] = 0
    for t in np.arange(n)[1:]:
        for v in range(len(tag)):
            for u in range(len(tag)):
                temp=[p[w, u, t - 1] * transmission_parameters(x, tag[w], tag[u], tag[v]) * emission_parameters(x, sent[t], tag[v]) for w in range(len(tag))]
                p[u, v, t] = np.max(temp)
                back[u, v, t] = np.argmax(temp)
    max_val=float('inf')            
    for v in range(len(tag)):
        for u in range(len(tag)):
            temp=p[u, v, n - 1]*transmission_parameters(x, tag[u], tag[v], 'STOP')
            if temp>max_val:
                z_pred[n-1]=v;z_pred[n-2]=u
    
    return backtrace(z_pred, back, n, tag)

In [6]:
def backtrace(z_pred, back, n, labels):
    prev1, prev2 = z_pred[-1], z_pred[-2]
    for k in np.arange(n-2)[::-1]:
        z_pred[k] = back[prev2, prev1,k+2]
    return [labels[z] for z in z_pred] 
        
def viterbi_ext(sent):
    #0- tag 'O'
    #1- tag 'I-GENE'
    n=len(sent)
    p = np.zeros(shape=(2, 2, n))
    back = np.zeros(shape=(2, 2, n), dtype=np.int)
    z_pred = np.zeros(shape=(n, ), dtype=np.int)
    tag=['O','I-GENE']
    for q in range(len(tag)):
        temp=transmission_parameters(x, '*', '*', tag[q]) * emission_parameters(x, sent[0], tag[q])
        p[q, 0, 0] = temp
        p[q, 1, 0] = temp
        back[q, 0, 0] = 0
        back[q, 1, 0] = 0
    for t in np.arange(n)[1:]:
        if t==n-1:
            next_word='STOP'
        else:
            next_word=sent[t+1]
        if emission_parameters(x, next_word, 'I-GENE') > emission_parameters(x, next_word, 'O'):
            next_tag='I-GENE'
        else:
            next_tag='O'
        for v in range(len(tag)):
            for u in range(len(tag)):                    
                temp=[p[w, u, t - 1] * transmission_parameters(x, tag[w], tag[u], tag[v]) * transmission_parameters_bigram(x, tag[v], next_tag)* emission_parameters(x, next_word, next_tag)* emission_parameters(x, sent[t], tag[v]) for w in range(len(tag))]
                p[u, v, t] = np.max(temp)
                back[u, v, t] = np.argmax(temp)
    max_val=float('inf')            
    for v in range(len(tag)):
        for u in range(len(tag)):
            temp=p[u, v, n - 1]*transmission_parameters(x, tag[u], tag[v], 'STOP')
            if temp>max_val:
                z_pred[n-1]=v;z_pred[n-2]=u
    
    return backtrace(z_pred, back, n, tag)

In [7]:
dev = open("gene.dev","r")
y=dev.readlines()
dev.close()

sentences=[];temp=[]
for i in y:
    i=i[:-1]
    if len(i)!=0:
        temp.append(i)
    if len(i)==0:
        sentences.append(temp);temp=[]    

In [8]:
dev = open("gene.train","r")
y=dev.readlines()
dev.close()

sentences=[];temp=[]
for i in y:
    i=i.split()
    if len(i)!=0:
        temp.append(i[0])
    if len(i)==0:
        sentences.append(temp);temp=[]

In [13]:
tags=[]
for i in sentences:
    if len(i)<=2:
        tags.append(['O','O'])
    else:
        tags.append(viterbi(i))

dev_test=open("gene_train_try1","w")
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        dev_test.write(sentences[i][j] + ' ' + tags[i][j]+ '\n')
    dev_test.write('\n')
dev_test.close()